In [1]:
import requests
import urllib.parse as up
from bs4 import BeautifulSoup
from selenium import webdriver
import datetime
import os
import pandas as pd

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:45.0) Gecko/20100101 Firefox/45.0'}  # 指定UA防止反爬
homepage = "https://tech.sina.com.cn/roll/"  # 滚动新闻

In [3]:
def getOneArticle(url):
    """
    获取一篇新闻的纯文本正文。
    Params:
        url: 新闻网址。
    Returns:
        由新闻标题、正文字符串组成的二元组。
    """
    re = requests.get(url)  # 下载页面
    re.raise_for_status()   # 若请求失败则抛出异常
    re.encoding = re.apparent_encoding  # 检测编码
    soup = BeautifulSoup(re.text)       # 解析HTML
    
    for s in soup('script'):
        s.extract()  # 丢弃HTML中的JS内容

    title = soup.select(".main-title")
    if len(title) == 0:
        title = soup.select("#artibodyTitle")
    title = title[0].text
    content = soup.find("div", id="artibody").text.strip()
    return title, content

In [4]:
def getPageLinks():
    """
    获取滚动新闻一个页面的所有新闻链接。
    Params:
        无。
    Returns:
        新闻链接的列表。
    """
    global browser
    results = browser.find_elements_by_xpath('//div[@class="d_list_txt"]/ul/li/span/a')
    links = []
    for result in results:
        links.append(result.get_attribute('href'))
    return links

In [5]:
counter = 0
metadata = []  # (id, 标题)元组的列表
visited = set()

browser = webdriver.Chrome()
browser.implicitly_wait(10)
browser.get(homepage)  # 滚动新闻首页

while True:
    links = getPageLinks()  # 获取滚动新闻一个页面的所有新闻链接
    for link in links:
        if link not in visited:  # 避免重复爬取
            visited.add(link)

            title, content = getOneArticle(link)

            if len(content) >= 1000:  # 忽略1000字以下的文章
                counter += 1
                metadata.append((counter, title))  # 记录id与标题的对应关系
                with open("data/" + str(counter) + ".txt", "w", encoding='utf-8') as f:
                    f.write(content)
                print(counter, title, len(content), link)

    browser.execute_script("newsList.page.next();return false;")  # 翻页
    browser.implicitly_wait(10)  # 等待页面加载完毕
    if counter >= 1000:
        break

browser.close()
print("[+] Done.")

1 央视快评：把区块链作为核心技术自主创新重要突破口 1137 https://tech.sina.com.cn/i/2019-10-26/doc-iicezuev5055058.shtml
2 生命在于“休息”？延年益寿秘诀在于大脑佛系思考 1225 https://tech.sina.com.cn/d/f/2019-10-26/doc-iicezuev5048295.shtml
3 抽卡、开箱、盲盒这类小把戏，是怎么操纵人心的？ 3277 https://tech.sina.com.cn/d/v/2019-10-26/doc-iicezzrr5090523.shtml
4 为什么有些场景明明是初次相遇，你却感到似曾相识？ 2272 https://tech.sina.com.cn/d/f/2019-10-26/doc-iicezuev5045359.shtml
5 大量蛙类面临灭绝危机 当务之急是教会它们谈恋爱 6033 https://tech.sina.com.cn/d/a/2019-10-26/doc-iicezzrr5086896.shtml
6 如果虫洞存在，要如何寻找？ 1282 https://tech.sina.com.cn/d/s/2019-10-26/doc-iicezuev5040868.shtml
7 丁磊当年要赶超百度的业务，十三年后终于在美国上市 6026 http://tech.sina.com.cn/csj/2019-10-26/doc-iicezuev5040098.shtml
8 小马智行现代在加州推自动驾驶出租车：居民免费乘坐 1718 https://tech.sina.com.cn/it/2019-10-26/doc-iicezzrr5082051.shtml
9 网易有道独立IPO：大公司为何偏爱分拆上市？ 3181 http://tech.sina.com.cn/csj/2019-10-26/doc-iicezuev5029229.shtml
10 世界青年科学家峰会本月26日在温开幕 1059 https://tech.sina.com.cn/d/i/2019-10-26/doc-iicezzrr5068148.shtml
11 孙宇晨：对于区块链造福每个中国人未来 我从未怀疑 1739 https://tech.

87 英特尔第三季度营收192亿美元 净利同比降6% 2186 https://tech.sina.com.cn/it/2019-10-25/doc-iicezzrr4783201.shtml
88 亚马逊第三季度营收700亿美元 净利同比降26% 2250 https://tech.sina.com.cn/i/2019-10-25/doc-iicezuev4738942.shtml
89 人民日报关注快递业：明星快递员 有啥不一般 2823 https://tech.sina.com.cn/i/2019-10-25/doc-iicezzrr4781306.shtml
90 人民日报：锂电池助推能源革新 1053 https://tech.sina.com.cn/it/2019-10-25/doc-iicezuev4738246.shtml
91 人民时评：网红经济诚信方有未来 1253 https://tech.sina.com.cn/i/2019-10-25/doc-iicezzrr4781246.shtml
92 扎克伯格6小时听证 议员：“更不相信脸书了” 2368 https://tech.sina.com.cn/i/2019-10-25/doc-iicezuev4737505.shtml
93 社交App围剿微信 垂直市场能否迎来生机？ 2516 https://tech.sina.com.cn/i/2019-10-25/doc-iicezuev4735413.shtml
94 支付巨头布局刷脸支付背后： 赔偿机制难消用户顾虑 2360 https://tech.sina.com.cn/i/2019-10-25/doc-iicezuev4734190.shtml
95 暗中涨价、红包陷阱…电商购物节是福利还是套路？ 1995 https://tech.sina.com.cn/i/2019-10-25/doc-iicezzrr4777320.shtml
96 家电业加速智能化、渠道变革 寻求新成长 2302 https://tech.sina.com.cn/e/2019-10-25/doc-iicezzrr4771499.shtml
97 社交电商贝店找500家厂牌启动新制造 1874 https://tech.sina.com.cn/i/2019-10-

173 谷歌量子计算突破，川普之女：美国实现量子霸权 5246 http://tech.sina.com.cn/csj/2019-10-24/doc-iicezzrr4567200.shtml
174 170岁“高龄”了 孪生素数猜想还未得到证明 2751 https://tech.sina.com.cn/d/i/2019-10-24/doc-iicezuev4450852.shtml
175 51信用卡被查引发"爬虫技术"争议 是否侵犯个人信息? 2246 https://tech.sina.com.cn/roll/2019-10-24/doc-iicezuev4520224.shtml
176 51信用卡称已终止催收外包 但仍在发布相关岗位 1921 https://tech.sina.com.cn/i/2019-10-24/doc-iicezuev4518616.shtml
177 媒体谈李国庆俞渝事件:再无往来 也比互爆隐私来得好 1868 https://tech.sina.com.cn/i/2019-10-24/doc-iicezuev4521249.shtml
178 火星和月球模拟土壤中可成功种植九种农作物 1182 https://tech.sina.com.cn/d/s/2019-10-24/doc-iicezzrr4559205.shtml
179 哎，好一个庆渝年！ 1493 http://tech.sina.com.cn/csj/2019-10-24/doc-iicezuev4514855.shtml
180 在互联网的世界，我们终将“老无所依” 3196 https://tech.sina.com.cn/i/2019-10-24/doc-iicezzrr4556119.shtml
181 量子霸权真的已经实现？IBM认为谷歌夸大成就 1175 https://tech.sina.com.cn/i/2019-10-24/doc-iicezuev4509084.shtml
182 别把李国庆俞瑜，跟贝佐斯麦肯齐放在一起比 3032 http://tech.sina.com.cn/csj/2019-10-24/doc-iicezzrr4550857.shtml
183 为什么Apple Maps黄了 5588 http://tech.sina.

258 三星承认旗舰手机存指纹漏洞 生物识别技术还能信？ 1791 https://tech.sina.com.cn/it/2019-10-23/doc-iicezuev4308316.shtml
259 双11要抢房！阿里放大招：上万套房特价卖 京东也5折 2124 https://tech.sina.com.cn/i/2019-10-23/doc-iicezzrr4332595.shtml
260 扎克伯格首次全面阐释天秤币：想把权力交给人们 6716 https://tech.sina.com.cn/i/2019-10-23/doc-iicezuev4276110.shtml
261 陈天桥 搬家到应许之地 14625 http://tech.sina.com.cn/csj/2019-10-23/doc-iicezzrr4306253.shtml
262 电子烟的“生死时速”:监管收紧下 电子烟也开始狂飙 2884 https://tech.sina.com.cn/i/2019-10-23/doc-iicezuev4247638.shtml
263 制定新法规能真正消除电子烟“危机”吗？ 1528 https://tech.sina.com.cn/it/2019-10-23/doc-iicezzrr4289914.shtml
264 国务院报告：2018年跨境电商零售进出口202.8亿美元 1465 https://tech.sina.com.cn/i/2019-10-23/doc-iicezuev4249838.shtml
265 玩游戏的老人们 11037 http://tech.sina.com.cn/csj/2019-10-23/doc-iicezzrr4274749.shtml
266 数千元可刷出“10万+” 流量造假助企业融资千万 2781 https://tech.sina.com.cn/i/2019-10-23/doc-iicezzrr4282395.shtml
267 OYO朱磊:下沉市场三大认知正被颠覆 2.0签约超5000家 1292 https://tech.sina.com.cn/i/2019-10-23/doc-iicezuev4229057.shtml
268 Microsoft的未来，靠Google？ 2863 http://

343 赵明：5G手机明年初真正放量，海外重点打造华为生态 1897 https://tech.sina.com.cn/5g/i/2019-10-22/doc-iicezzrr4133727.shtml
344 传音控股刚科创板上市就拿25亿现金理财 被指“奇葩” 1303 http://tech.sina.com.cn/csj/2019-10-22/doc-iicezuev4088740.shtml
345 数字人才流向哪些城市？深圳吸引力最强 1277 https://tech.sina.com.cn/it/2019-10-22/doc-iicezzrr4124996.shtml
346 思科全球首席创新官：AI、5G和IoT等技术惠及大众 2742 https://tech.sina.com.cn/it/2019-10-22/doc-iicezuev4076834.shtml
347 杭州总部被查后 51信用卡上海分部员工所剩无几 3400 https://tech.sina.com.cn/i/2019-10-22/doc-iicezzrr4146785.shtml
348 洛阳警方打掉全国性二维码诈骗组织 涉案额3000多万 1270 https://tech.sina.com.cn/i/2019-10-22/doc-iicezzrr4144675.shtml
349 51信用卡终止催收外包 律师:骚扰借款人亲友构成侵权 1311 https://tech.sina.com.cn/i/2019-10-22/doc-iicezzrr4109640.shtml
350 99元游马尔代夫 “旅游尾单群”是馅饼还是陷阱？ 4404 https://tech.sina.com.cn/i/2019-10-22/doc-iicezzrr4110018.shtml
351 51信用卡用户致电催收人不要骚扰通讯录亲友 但无效 1196 https://tech.sina.com.cn/i/2019-10-22/doc-iicezuev4061458.shtml
352 宝马与戴姆勒合资出行公司寻“新欢” 降低运营风险 2250 https://tech.sina.com.cn/it/2019-10-22/doc-iicezuev4095988.shtml
353 对话美图CE

...

599 陆奇：创新推动社会进步 技术是创新最持久的驱动力 2030 https://tech.sina.com.cn/i/2019-10-20/doc-iicezzrr3510873.shtml
600 脉脉林凡：希望"职言"解决真问题 而不是成爆料平台 3586 http://tech.sina.com.cn/csj/2019-10-20/doc-iicezuev3464875.shtml
601 沈向洋：微软每一个技术研发都会进行AI伦理道德评审 1761 https://tech.sina.com.cn/i/2019-10-20/doc-iicezuev3462569.shtml
602 阿里巴巴张勇谈新商业文明：让更多小草长成参天大树 3933 https://tech.sina.com.cn/i/2019-10-20/doc-iicezuev3460867.shtml
603 李彦宏乌镇大会首提智能经济 称AI可让人获“永生” 3189 https://tech.sina.com.cn/i/2019-10-20/doc-iicezuev3456525.shtml
604 黄坤明宣读习近平主席贺信并发表主旨演讲 1102 https://tech.sina.com.cn/i/2019-10-20/doc-iicezuev3460427.shtml
605 这引人联想的机械右手！玩转魔方，灵活不输人类 3687 http://tech.sina.com.cn/csj/2019-10-20/doc-iicezzrr3493648.shtml
606 结构性存款大变局:支付宝微信京东App代销或终结? 2211 https://tech.sina.com.cn/i/2019-10-20/doc-iicezzrr3490535.shtml
607 贾跃亭“破产”：总有一笔债，是赖不掉的 4036 https://tech.sina.com.cn/i/2019-10-20/doc-iicezuev3439583.shtml
608 关厂裁员？实地探访知名手机设计公司海派通讯 3763 https://tech.sina.com.cn/it/2019-10-20/doc-iicezzrr3479624.shtml
609 任正非说到做到！华为正与美企谈判授权转让5G技术

685 一加将在印度建第二总部 四季度会有双模5G手机 1945 https://tech.sina.com.cn/5g/product/2019-10-18/doc-iicezuev3156374.shtml
686 全球近八成运营商将部署SA模式5G网络 4G仍将共存 1239 https://tech.sina.com.cn/5g/i/2019-10-18/doc-iicezzrr3201022.shtml
687 Uber上市5月启动三轮裁员 被曝考虑终止自动驾驶项目 1223 https://tech.sina.com.cn/i/2019-10-18/doc-iicezzrr3211735.shtml
688 探访丽泽SOHO神秘31层：5G实验室细节曝光|5G Weekly 2222 https://tech.sina.com.cn/5g/i/2019-10-18/doc-iicezuev3122678.shtml
689 VIPKID成立六周年 米雯娟内部信：坚持做正确的事 1977 https://tech.sina.com.cn/i/2019-10-18/doc-iicezzrr3161671.shtml
690 邬贺铨：“全方位开放合作”包括互联网的开放合作 1892 https://tech.sina.com.cn/i/2019-10-20/doc-iicezzrr3466778.shtml
691 蔚来、威马、小鹏立下的年销目标 今年几乎已告败 3424 https://tech.sina.com.cn/it/2019-10-18/doc-iicezuev3102837.shtml
692 小米集团创始人雷军获复旦企业管理杰出贡献奖 1885 https://tech.sina.com.cn/i/2019-10-18/doc-iicezuev3111925.shtml
693 中国科学家成功解析非洲猪瘟病毒结构 助新疫苗开发 1491 https://tech.sina.com.cn/d/f/2019-10-18/doc-iicezuev3080210.shtml
694 报告：中国5G产业领跑全球 已跻身第一阵营 1256 https://tech.sina.com.cn/5g/i/2019-10-18/doc-iicezzrr313177

770 圆通探路中高端市场受挫 承诺达运营1年直营改加盟？ 2413 https://tech.sina.com.cn/it/2019-10-18/doc-iicezzrr2984519.shtml
771 负债百亿的贾跃亭收入曝光：一个月收入能买一辆奔驰 2049 https://tech.sina.com.cn/it/2019-10-18/doc-iicezuev2938952.shtml
772 不到银行也能存款 微信联手工行上线“银行储蓄” 1753 https://tech.sina.com.cn/i/2019-10-18/doc-iicezzrr2984560.shtml
773 蔚来洽谈超50亿融资？产业基金投新能源车尴尬承压 2259 https://tech.sina.com.cn/i/2019-10-18/doc-iicezzrr2984549.shtml
774 网贷整治时间表明确了 明年上半年完成存量风险化解 2676 https://tech.sina.com.cn/i/2019-10-18/doc-iicezzrr2984556.shtml
775 趣步涉嫌传销被查“躺赚”App真的可以躺赚吗? 2439 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2925382.shtml
776 全球最大暗网儿童色情网站被捣毁 暗网到底是啥？ 1563 https://tech.sina.com.cn/i/2019-10-18/doc-iicezuev2924148.shtml
777 自燃、“趴窝”、没补贴……新能源车原来那么惨！ 4913 https://tech.sina.com.cn/it/2019-10-18/doc-iicezuev2967752.shtml
778 阅文与迪士尼合作首部“星战”中文小说 盘中涨超8% 1141 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2924117.shtml
779 外卖员往食物里吐4次口水 顺丰:系供应商员工 已封号 1165 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2914284.shtml
780 融合公域私域社交资源 微博

854 人民日报海外版：“互联网+留学”简化办证流程 1333 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2707539.shtml
855 孙正义的成败画像:被Uber和WeWork套牢 非第一次摔跤 2950 https://tech.sina.com.cn/i/2019-10-17/doc-iicezzrr2753075.shtml
856 淘集集自救:数千家供应商被拖欠货款 变身股东合伙人 2689 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2704598.shtml
857 国内不宜盲目引进落后液晶产能 1610 https://tech.sina.com.cn/it/2019-10-17/doc-iicezzrr2750044.shtml
858 飞猪推新店运营体系 总裁赵颖：赋能商家数字化转型 1062 https://tech.sina.com.cn/i/2019-10-17/doc-iicezzrr2749898.shtml
859 “合伙人制度”救得了淘集集吗 1355 https://tech.sina.com.cn/i/2019-10-17/doc-iicezzrr2746066.shtml
860 量子波动速读何以得逞？背后是挥之不去的教育焦虑 1198 https://tech.sina.com.cn/it/2019-10-17/doc-iicezuev2699189.shtml
861 北青报：电商平台“二选一”问题该有权威认定了 1331 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2699185.shtml
862 在线旅游消费捆绑搭售猫腻多 1229 https://tech.sina.com.cn/i/2019-10-17/doc-iicezuev2699174.shtml
863 学生在校玩手机 没收后被销毁 1462 https://tech.sina.com.cn/it/2019-10-17/doc-iicezzrr2744706.shtml
864 冲击“知识付费第一股” 罗辑思维备战科创板 1532 https://tech.sina.com.cn/

940 语C、耽美、盲盒、Lolita……这些95后文化你懂吗？ 5227 http://tech.sina.com.cn/csj/2019-10-16/doc-iicezzrr2542400.shtml
941 开源简史：从社区到商业化 10926 http://tech.sina.com.cn/csj/2019-10-16/doc-iicezzrr2538428.shtml
942 人类史上十个最重要实验:测量地球周长 牛顿发展光学 8090 https://tech.sina.com.cn/d/v/2019-10-16/doc-iicezuev2491828.shtml
943 贾跃亭20大债权人详解：英大资本索赔2.79亿美元居首 3047 https://tech.sina.com.cn/i/2019-10-16/doc-iicezuev2492616.shtml
944 索尼即将推出360 Reality Audio 提供沉浸式音乐体验 1302 https://tech.sina.com.cn/e/2019-10-16/doc-iicezuev2489425.shtml
945 背靠阿里腾讯 墨迹天气为何难上市？ 3062 https://tech.sina.com.cn/i/2019-10-16/doc-iicezzrr2639813.shtml
946 人类头骨遵循“黄金比例1.618”？解剖学家称荒谬 1422 https://tech.sina.com.cn/d/f/2019-10-16/doc-iicezzrr2535562.shtml
947 三星手机重返中国失利：关闭本地工厂 市场份额仍1% 3379 https://tech.sina.com.cn/t/2019-10-16/doc-iicezzrr2533744.shtml
948 Google硬件全家桶大更新！槽点简直和亮点一样多 6938 http://tech.sina.com.cn/csj/2019-10-16/doc-iicezuev2486900.shtml
949 谷歌发布一揽子硬件：策略日渐清晰 强调"环境计算" 2139 https://tech.sina.com.cn/t/2019-10-16/doc-iicezzrr2534079.shtml
950 VRChat

In [10]:
# 把新闻id及标题的对应关系写入csv文件（这一步是可选的，后续工作并未用到该信息）
df = pd.DataFrame(metadata, columns=["id", "title"])
df.to_csv("metadata.csv", index=False, encoding="utf-8")